In [1]:
# %run ../../chap01/codes/implement.ipynb

In [2]:
def init_model_hidden1():
    global pm_output, pm_hidden
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
    
def alloc_param_pair(shape):
    weight = np.random.normal(_rnd_mean, _rnd_std, shape)
    bias = np.zeros(shape[-1])
    return {'w': weight, 'b': bias}


In [3]:
def forward_neuralnet_hidden1(x):
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    return output, [x, hidden]


def relu(x):
    return np.maximum(x, 0)


In [4]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    x, hidden = aux
    
    # 출력층 그래디언트
    g_output_w_out = hidden.transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)
    
    # 은닉층 그래디언트를 구하기위해 출력층 가중치들을 저장
    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= _learning_rate * G_w_out
    pm_output['b'] -= _learning_rate * G_b_out
    
    G_hidden = G_hidden * relu_derv(hidden)
    g_hidden_w_hid = x.transpose()
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
    G_b_hid = np.sum(G_hidden, axis=0)
    
    pm_hidden['w'] -= _learning_rate * G_w_hid
    pm_hidden['b'] -= _learning_rate * G_b_hid
    
    

def relu_derv(y):
    return np.sign(y)


In [5]:
def init_model_hiddens():
    global pm_output, pm_hiddens
    
    pm_hiddens = []
    prev_cnt = input_cnt

    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
        
    pm_output = alloc_param_pair([prev_cnt, output_cnt])
    

In [6]:
def forward_neuralnet_hiddens(x):
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

In [7]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens
    hiddens = aux  # 가독성의 이유로...
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)
    
    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= _learning_rate * G_w_out
    pm_output['b'] -= _learning_rate * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])
        
        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
        
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= _learning_rate * G_w_hid
        pm_hiddens[n]['b'] -= _learning_rate * G_b_hid


In [10]:
def init_model():
    if hidden_config is not None:
        print("은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.".format(len(hidden_config)))
        init_model_hiddens()
    else:
        print("은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.")
        init_model_hidden1()
        

def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)

In [9]:
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info
        